In [2]:
import sys

sys.path.append("../../")

%load_ext autoreload
%autoreload 2

In [22]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import hashlib
from ast import literal_eval
from pathlib import Path
from functools import reduce

from src.utils import find_meta_category
from src.feature_extractor import sample_feature_combinations

## Download prerequisite files

Fetch all the results and feature values


In [4]:
# You can get the experiments file here: 01J6KF3JRCATRJQ9CPJTRV5VBM (https://beaker.org/ds/01J6KF3JRCATRJQ9CPJTRV5VBM/details)
!echo "Fetching experiments list..."
!beaker dataset fetch 01J6KF3JRCATRJQ9CPJTRV5VBM --prefix experiments.txt
!echo "Fetching extracted features..."
!mkdir features/
!beaker dataset fetch 01J6KF3JRCATRJQ9CPJTRV5VBM --prefix features/ 
#!beaker dataset fetch 01J6KFVCRCTYHCZDR0XNK0G9HT --prefix features/
!echo "Fetching helpsteer2 dataset"
!beaker dataset fetch 01J6KBM2VCM9EQ7MER26VBXCCM
!echo "Collating all evaluation results"
%run ../../scripts/fetch_evals_rewardbench.py --output_file results.csv --gpt4_threshold_score 0.658 --experiment_prefix rm-eval-helpsteer2 --experiments_file experiments.txt

Fetching experiments list...
Files: 0          ⠋  
Bytes: 0 B        ⠋  
Files: 1          ⠙  
Bytes: 73.77 KiB  ⠙  
Files: 1          ✔  
Bytes: 73.77 KiB  ✔  
Files: 1          ✔  
Bytes: 73.77 KiB  ✔  
Completed in 100ms: 439.4 KiB/s, 6 files/s
Fetching extracted features...
mkdir: features/: File exists
Files: 0          ⠋  
Bytes: 0 B        ⠋  
Files: 2          ⠙  
Bytes: 75.1 MiB   ⠙  
Files: 9          ⠹  
Bytes: 339 MiB    ⠹  
Files: 16         ⠸  
Bytes: 602.9 MiB  ⠸  
Files: 16         ✔  
Bytes: 602.9 MiB  ✔  
Files: 16         ✔  
Bytes: 602.9 MiB  ✔  
Completed in 400ms: 1.23 GiB/s, 33 files/s
Fetching helpsteer2 dataset
Files: 0          ⠋  
Bytes: 0 B        ⠋  
Files: 1          ⠙  
Bytes: 70.58 MiB  ⠙  
Files: 1          ✔  
Bytes: 70.58 MiB  ✔  
Files: 1          ✔  
Bytes: 70.58 MiB  ✔  
Completed in 100ms: 362.2 MiB/s, 5 files/s
Collating all evaluation results
2024-09-02 12:59:53 - INFO - root - Logged-in as ljm (ljm@allenai.org)
2024-09-02 12:59:54 - INFO - root

Collate feature set for all instances


In [5]:
LEXICAL_FEATS_PATH = Path("features")
DATASET_PATH = Path("helpsteer2_human_vs_gpt4_weighted_for_llama.jsonl")


def get_dataset_features(
    feature_path=LEXICAL_FEATS_PATH, dataset_path=DATASET_PATH
) -> "pd.DataFrame":
    lexical_features = [
        "rouge",
        "bertscore",
        "bertscore_length",
        "entity_sim",
        "cosine_sim",
        "prompt_len",
        "len_longer",
        "len_shorter",
        "token_len_difference",
    ]
    lexical_feature_files = [
        file
        for file in feature_path.glob("*.jsonl")
        if any(file.stem in feat for feat in lexical_features)
    ]
    lexical_feats_df = reduce(
        lambda left, right: left.merge(
            right, on=["id", "prompt", "completion_a", "completion_b"], how="outer"
        ),
        [pd.read_json(file, lines=True) for file in lexical_feature_files],
    )

    df = pd.read_json(dataset_path, lines=True).rename(columns={"prompt_hash": "id"})
    finaldf = df.merge(lexical_feats_df, how="left", on="id").drop(
        columns=["prompt", "completion_a", "completion_b"]
    )

    # Hacky way for token_len_difference
    finaldf = finaldf.rename(columns={"token_len_diff": "token_len_difference"})
    return finaldf

In [6]:
results_df = pd.read_csv("results.csv").dropna()
features_df = get_dataset_features()
print(len(results_df))

62


## Get proportion of instances that fulfill the conditions

1. For each row, get features that were activated
2. Then for each activated feature, we get the proportion by looking at the feature dataframe.
3. The proportion is computed as: `number_of_instance_that_fulfill_a_single_condition` / `total_number_of_instances`


In [7]:
# Inspect nan columns
rows_with_nan = features_df[features_df.isna().any(axis=1)]
nan_columns = rows_with_nan.columns[rows_with_nan.isna().any()]
df_nan_columns = rows_with_nan[nan_columns]
df_nan_columns

,expertise_level,format_constraints
289,None,[]
1317,expert domain knowledge,None
4613,basic domain knowledge,None
4734,general public,None


So what you're going to do instead, is to take the binary_cols, and then for each element of that binary_cols, you compute the "weight"


In [8]:
def compute_instances(feat: str, features_df: "pd.DataFrame") -> float:
    total = len(features_df)
    lexical_features = [
        "rouge",
        "bertscore",
        "bertscore_length",
        "entity_sim",
        "cosine_sim",
        "prompt_len",
        "len_longer",
        "len_shorter",
        "token_len_difference",
    ]

    if feat.split("__")[0] in lexical_features:
        feat_name, value = feat.split("__")
        min_val_str, max_val_str = value.split("|")
        min_val, max_val = float(min_val_str.split("=")[1]), float(
            max_val_str.split("=")[1]
        )
        return features_df[feat_name].between(min_val, max_val).mean()
    else:
        # Parse the feature
        feat_name, value = feat.split("=")
        meta_category = find_meta_category(feat_name)
        if meta_category == "scalar":
            v = value.replace("_", " ")
            return features_df[feat_name].value_counts().get(v) / total
        elif meta_category == "closed_set":
            v = value.replace("_", " ")
            list_of_values = features_df[feat_name].tolist()
            return sum([1 if v in listval else 0 for listval in list_of_values]) / total
        elif meta_category == "open_set":
            list_of_values = features_df[feat_name].tolist()
            return sum([1 if listval else 0 for listval in list_of_values]) / total

        return find_meta_category(feat_name)


feats = results_df.columns[results_df.isin([0, 1]).all()]  # get binary columns
feat_map = {
    feat: compute_instances(feat, features_df) for feat in feats if feat != "label"
}

ratio_df = results_df.apply(
    lambda row: row.map(lambda x: feat_map.get(row.name, 1) if x == 1 else x)
)

# Regressor training


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

## Train LightGBM regressor


In [10]:
params = {
    "objective": "regression",
    "metric": "mse",
    "boosting_type": "gbdt",
    "learning_rate": 0.1,
    "num_leaves": 31,
}

# Train the model
binary = False
X = ratio_df[list(feat_map.keys())]
if binary:
    X = (X > 0).astype(int)
y = ratio_df["Overall"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)
print(f"Train size: {len(X_train)}, test size: {len(X_test)}")


train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
model = lgb.train(params, train_data, valid_sets=[test_data])

# Predict and evaluate
y_pred = model.predict(X_test, num_iteration=model.best_iteration)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Train size: 55, test size: 7
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 55, number of used features: 0
[LightGBM] [Info] Start training from score 0.695253
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no m

In [11]:
importances = model.feature_importance(importance_type="gain")  # ['split', 'gain']

# Create a DataFrame to view feature importances
feature_importance_df = pd.DataFrame(
    {"Feature": X.columns, "Importance": importances}
).sort_values(by="Importance", ascending=False)

print(feature_importance_df)

                                  Feature  Importance
0    bertscore__min_val=0.33|max_val=0.67         0.0
33       rouge__min_val=0.33|max_val=0.67         0.0
35                    safety_concern=high         0.0
36                     safety_concern=low         0.0
37                safety_concern=moderate         0.0
..                                    ...         ...
27                open_endedness=moderate         0.0
28                      open_endedness=no         0.0
29   prompt_len__min_val=0.0|max_val=0.33         0.0
30  prompt_len__min_val=0.33|max_val=0.67         0.0
64          type_of_in_context_material=1         0.0

[65 rows x 2 columns]


## Train LinearRegressor


In [12]:
polyfit = True
binary = False

X = ratio_df[list(feat_map.keys())]
y = ratio_df["Overall"]
if binary:
    X = (X > 0).astype(int)

if polyfit:
    poly = PolynomialFeatures(degree=2, include_bias=False)
    X_poly = poly.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(
        X_poly, y, test_size=0.2, random_state=42
    )
else:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )


print(f"Train size: {len(X_train)}, test size: {len(X_test)}")

model = LinearRegression()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Feature names: {poly.get_feature_names_out(X.columns)}")
print(f"Mean Squared Error: {mse}")
print(f"Coeeficients: {model.coef_}")
print(f"Intercept: {model.intercept_}")

Train size: 49, test size: 13
Feature names: ['bertscore__min_val=0.33|max_val=0.67'
 'bertscore__min_val=0.67|max_val=1.0'
 'bertscore_length__min_val=0.0|max_val=0.33' ...
 'token_len_difference__min_val=0.67|max_val=1.0^2'
 'token_len_difference__min_val=0.67|max_val=1.0 type_of_in_context_material=1'
 'type_of_in_context_material=1^2']
Mean Squared Error: 0.0012591021224480326
Coeeficients: [-4.07087444e-01  2.64518756e-02  7.84504190e-02 ...  1.13884130e-03
  0.00000000e+00  3.17208325e-04]
Intercept: 0.7141910311097194


In [13]:
if not polyfit:
    feature_importance = pd.DataFrame(
        {"Feature": X.columns, "Coefficient": model.coef_}
    )

    # Calculate absolute importance for easier comparison
    feature_importance["Absolute_Coefficient"] = np.abs(
        feature_importance["Coefficient"]
    )

    # Sort by absolute coefficient value
    feature_importance = feature_importance.sort_values(
        by="Absolute_Coefficient", ascending=False
    )
    feature_importance.head(10)
else:
    print(
        "Feature importance is not possible with polynomial features (hard to interpret)"
    )

Feature importance is not possible with polynomial features (hard to interpret)


## Simulation


In [14]:
from tqdm import tqdm_notebook

In [15]:
_, combinations = sample_feature_combinations(
    meta_analyzer_n_samples=2000, max_number=10
)

10it [00:00, 46500.04it/s]
45it [00:00, 68609.12it/s]
120it [00:00, 62859.56it/s]
210it [00:00, 50467.19it/s]
252it [00:00, 49643.73it/s]
210it [00:00, 41807.66it/s]
120it [00:00, 34911.32it/s]
45it [00:00, 30795.18it/s]
10it [00:00, 20794.76it/s]
1it [00:00, 8924.05it/s]

2024-09-02 13:00:21 - INFO - root - Adding meta analyzer features



10it [00:00, 110960.42it/s]
45it [00:00, 106936.93it/s]
120it [00:00, 75527.68it/s]
210it [00:00, 62015.34it/s]
252it [00:00, 48155.48it/s]
210it [00:00, 41389.21it/s]
120it [00:00, 35037.69it/s]
45it [00:00, 29217.29it/s]
10it [00:00, 21421.37it/s]
1it [00:00, 15650.39it/s]


In [16]:
sim_df = pd.DataFrame(0, index=np.arange(len(combinations)), columns=X.columns)
for idx, combination in tqdm_notebook(enumerate(combinations), total=len(combinations)):
    activated_feats = []
    for feat in combination:
        if "analyzer" in feat:
            feature_name_str, value_str = feat.split("::")[1].split("|")
            feature_name, value = (
                feature_name_str.split("=")[-1],
                value_str.split("=")[-1],
            )
            activated_feats.append(f"{feature_name}={value}")
        else:
            activated_feats.append(feat.replace("::", "__"))
    sim_df.loc[idx, activated_feats] = 1
sim_df = sim_df.apply(
    lambda row: row.map(lambda x: feat_map.get(row.name, 1) if x == 1 else x)
).dropna(axis=1, how="any")

/var/folders/hx/gk2rs0792pn5p8hkj4nkhdm80000gp/T/ipykernel_36621/4023816742.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, combination in tqdm_notebook(enumerate(combinations), total=len(combinations)):


  0%|          | 0/4069 [00:00<?, ?it/s]

In [17]:
sim_results = sim_df.copy(deep=True)
sim_results["activated_features"] = sim_results.apply(
    lambda row: [col for col in sim_results.columns if row[col] != 0], axis=1
)
sim_results["pred"] = model.predict(poly.transform(sim_df))
sim_results = sim_results.sort_values(by="pred", ascending=False).reset_index(drop=True)
sim_results[["activated_features", "pred"]].head(20)

,activated_features,pred
0,"[bertscore__min_val=0.67|max_val=1.0, complexi...",0.812256
1,"[bertscore_length__min_val=0.0|max_val=0.33, e...",0.810126
2,"[complexity_of_intents=simple, safety_concern=...",0.800891
3,"[bertscore__min_val=0.67|max_val=1.0, cosine_s...",0.799207
4,"[bertscore__min_val=0.67|max_val=1.0, cosine_s...",0.798044
5,"[bertscore_length__min_val=0.33|max_val=0.67, ...",0.792909
6,"[complexity_of_intents=simple, format_constrai...",0.791309
7,"[complexity_of_intents=simple, format_constrai...",0.791309
8,"[complexity_of_intents=simple, format_constrai...",0.790966
9,"[complexity_of_intents=simple, format_constrai...",0.790966


In [18]:
top_combinations = sim_results.activated_features.head(10).to_list()
print(top_combinations)

[['bertscore__min_val=0.67|max_val=1.0', 'complexity_of_intents=simple', 'entity_sim__min_val=0.33|max_val=0.67', 'token_len_difference__min_val=0.67|max_val=1.0', 'type_of_in_context_material=1'], ['bertscore_length__min_val=0.0|max_val=0.33', 'entity_sim__min_val=0.33|max_val=0.67', 'open_endedness=moderate', 'prompt_len__min_val=0.67|max_val=1.0', 'safety_concern=safe'], ['complexity_of_intents=simple', 'safety_concern=safe'], ['bertscore__min_val=0.67|max_val=1.0', 'cosine_sim__min_val=0.67|max_val=1.0', 'entity_sim__min_val=0.0|max_val=0.33', 'token_len_difference__min_val=0.67|max_val=1.0'], ['bertscore__min_val=0.67|max_val=1.0', 'cosine_sim__min_val=0.67|max_val=1.0', 'entity_sim__min_val=0.0|max_val=0.33'], ['bertscore_length__min_val=0.33|max_val=0.67', 'complexity_of_intents=simple', 'format_constraints=1', 'open_endedness=no', 'rouge__min_val=0.0|max_val=0.33', 'safety_concern=safe', 'type_of_in_context_material=1'], ['complexity_of_intents=simple', 'format_constraints=1', 

TODO: So now you have determined 10 feature combinations that seem to work well. The next step is to train RMs and evaluate them.


In [23]:
for combination in top_combinations:
    feats_to_run = []
    for feat in combination:
        if "min_val" in feat:
            feats_to_run.append(feat.replace("__", "::"))
        else:
            feat_name, value = feat.split("=")
            category = find_meta_category(feat_name)
            if category == "closed_set":
                key = "constraints"
            elif category == "scalar":
                key = "value"
            elif category == "open_set":
                key = "check_for_existence"
            feats_to_run.append(f"{category}::feature_name={feat_name}|{key}={value}")
    features_str = " ".join(feats_to_run)
    print(features_str)
    %run ../../scripts/apply_data_model.py single --input_path helpsteer2_human_vs_gpt4_weighted_for_llama.jsonl --output_dir . --append_to_experiments_file top_features_experiments.txt --features {features_str}

bertscore::min_val=0.67|max_val=1.0 scalar::feature_name=complexity_of_intents|value=simple entity_sim::min_val=0.33|max_val=0.67 token_len_difference::min_val=0.67|max_val=1.0 open_set::feature_name=type_of_in_context_material|check_for_existence=1


10it [00:00, 88674.50it/s]
45it [00:00, 98663.71it/s]
120it [00:00, 80750.28it/s]
210it [00:00, 59225.65it/s]
252it [00:00, 51895.94it/s]
210it [00:00, 41160.98it/s]
120it [00:00, 35565.04it/s]
45it [00:00, 32728.23it/s]
10it [00:00, 20784.46it/s]
1it [00:00, 15141.89it/s]

2024-09-02 13:12:15 - INFO - root - Adding meta analyzer features



10it [00:00, 91779.08it/s]
45it [00:00, 104857.60it/s]
120it [00:00, 74400.07it/s]
210it [00:00, 59658.89it/s]
252it [00:00, 49774.65it/s]
210it [00:00, 40439.09it/s]
120it [00:00, 33831.85it/s]
45it [00:00, 29376.45it/s]
10it [00:00, 22721.04it/s]
1it [00:00, 15141.89it/s]


2024-09-02 13:12:15 - INFO - root - Found 10160 prompts with cols: ['prompt_hash', 'text', 'response_a', 'response_b', 'pref_human', 'pref_gpt4', 'rating_human', 'rating_gpt4', 'completions', 'subject_of_expertise', 'expertise_level', 'languages', 'open_endedness', 'safety_concern', 'complexity_of_intents', 'type_of_in_context_material', 'format_constraints']
2024-09-02 13:12:15 - INFO - root - Using device: cpu
2024-09-02 13:12:15 - INFO - root - Extracting features
2024-09-02 13:12:15 - INFO - root - Extracting 'bertscore' with params: {'min_val': 0.67, 'max_val': 1.0}


/Users/ljvm/Documents/Work/human-pref-datamodel/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


calculating scores...
computing bert embedding.


  0%|          | 0/317 [00:00<?, ?it/s]